<a href="https://colab.research.google.com/github/JacobJS56/FootballRaterService/blob/main/Football_Webscraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre

In [11]:
import requests
import time
import re
import json
import os
from bs4 import BeautifulSoup
from google.colab import drive
drive.mount('/content/gdrive')

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [12]:
URL_PREFIX = "https://www.footballdatabase.eu"

# Teams

In [13]:
URL = URL_PREFIX + "/en/competition/overall/14983-la_liga_santander/2021-2022"
page = requests.get(URL)
league = "La Liga Santander"
main_soup = BeautifulSoup(page.content, "html.parser")

In [14]:
teamList = main_soup.find_all("div", class_="row")
teamList = main_soup.find_all("div", class_="module ranking fullsize")
teamList = main_soup.find_all("td", class_="club")
teamList = teamList[0:20]

In [15]:
team_dict = dict()

for team in teamList:
  team_name = team.text.strip()
  team = team.find("a", class_="")
  team_url = team.get('href')
  team_dict[team_name] = team_url

# Players

Gets all players from all teams

In [ ]:
teams_dict_withplayers = dict()
listOfTeams= []

for key in team_dict:
 # time.sleep(25)
  hashedKey = str(abs(hash(key)) % (10 ** 8))

  URL = URL_PREFIX + team_dict[key]
  page = requests.get(URL)
  team_soup = BeautifulSoup(page.content, "html.parser")

  listOfPlayers = []
  teamD = dict()

  tsuc=False
  ssuc=False
  msuc=False

  # logo
  try:
    logo = team_soup.find("div", class_="clublogo")
    for l in logo:
      with open('/content/gdrive/My Drive/football/logos/' + hashedKey + '-logo.png', 'wb') as f:
        f.write(requests.get(URL_PREFIX + str(l["src"])).content)
    tsuc=True
  except:
    print("Couldn't get Logo for: %s", key)
  # time to update the drive
  time.sleep(25)

  teamD["teamName"] = key

  # stadium
  try:
    stadium = team_soup.find("div", class_="info")
    stadium = str(stadium).split("<div>")[5]
    stadium = re.sub("</div>", "", str(stadium))
    stadium = re.sub("(.*)span>", "", str(stadium))
    teamD["stadium"] = stadium
    ssuc=True
  except:
    print("Couldn't get Stadium for: %s", key)

  # manager
  try:
    manager = team_soup.find("div", class_="info-manager")
    manager.find("p", class_="manager")
    manager = re.sub("Manager(.*)", "", str(manager.text.strip()))
    manager = manager.split("\n")[0]
    teamD["manager"] = manager
    msuc=True
  except:
    print("Couldn't get Manager for: %s", key)

  # find the logos shareable link
  for d in drive.ListFile({'q': "'1MSknsWLdfdb7yFZ906uT4JZOA4-L5Ni6' in parents and trashed=false"}).GetList():
    if(d['originalFilename'] == hashedKey + '-logo.png'):
      teamD["logo"] = d['alternateLink']

  # players
  # new folder for players of particular team
  player_path = "/content/gdrive/My Drive/football/players/" + key + "/"
  os.mkdir(player_path)

  # go through list of players
  playerList = team_soup.find_all("div", class_="headband-player")
  player_name = ""

  for player in playerList:
    # find players name
    player_name = player.find("h3", class_="name")
    # hash player name with key
    player_name_hashed = str(abs(hash(str(key) + str(player_name.text.strip()))) % (10 ** 16))
    # get picture for player
    player_image = player.find("img")
    # save picture
    with open(player_path + player_name_hashed + '.png', 'wb') as f:
        f.write(requests.get(URL_PREFIX + player_image['src']).content)
    listOfPlayers.append(player_name.text.strip())
  # time to update the drive
  time.sleep(25)

  # get shareable link for players
  newListOfPlayers = []
  # find folder with team name
  for d in drive.ListFile({'q': "'103_ikqARGHGPvr0f1TnL2fLPQSsK3mxB' in parents and trashed=false"}).GetList():
    if(d['title'] == key):
      # iterate through players in listOfPlayers
      for pl in listOfPlayers:
        # rehash to check file name
        player_name_hashed = str(abs(hash(str(key) + str(pl))) % (10 ** 16))
        # iterate files to find a match
        for f in drive.ListFile({'q': f"'{d['id']}' in parents"}).GetList():
          if(player_name_hashed + '.png' == f['originalFilename']):
            # obtain shareable link
            newListOfPlayers.append({'name': pl, 'image':  f['alternateLink']})
  
  # players list added to dict and added to teams list
  teamD["playerList"] = newListOfPlayers
  listOfTeams.append(teamD)

  # final check make sure successful
  if(ssuc and msuc and tsuc):
    print("Succesful for: " + key)

# league dict creation
teams_dict_withplayers["league"] = league
teams_dict_withplayers["teamList"] = listOfTeams

In [ ]:
print(json.dumps(teams_dict_withplayers, indent=4))

In [18]:
jsonString = json.dumps(teams_dict_withplayers, indent=4)
with open('/content/gdrive/My Drive/football/' + league + '.json', 'w') as f:
  f.write(jsonString)